In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D7036A5C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D70369360>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hi Krish, it's nice to meet you! That's a fascinating role.  \n\nWhat kind of AI projects are you working on these days?  I'm always eager to learn more about the exciting things people are doing with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 22, 'total_tokens': 76, 'completion_time': 0.098181818, 'prompt_time': 0.00014557, 'queue_time': 0.014171268, 'total_time': 0.098327388}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b224366-c535-49f8-8439-b5c3e438be26-0', usage_metadata={'input_tokens': 22, 'output_tokens': 54, 'total_tokens': 76})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="According to our conversation, your name is Krish and you are a Chief AI Engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 97, 'total_tokens': 139, 'completion_time': 0.076363636, 'prompt_time': 0.003458156, 'queue_time': 0.010914611999999999, 'total_time': 0.079821792}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8fa2e65f-72fe-48b4-b406-309fde4e3f13-0', usage_metadata={'input_tokens': 97, 'output_tokens': 42, 'total_tokens': 139})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [14]:
response.content

"Hi Krish, it's nice to meet you! \n\nBeing a Chief AI Engineer is a fascinating role. What are some of the most exciting projects you're working on right now? \n\nI'm always eager to learn more about the cutting edge of AI development.  \n\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  😊 \n\nI remembered from our earlier conversation!  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 107, 'total_tokens': 135, 'completion_time': 0.050909091, 'prompt_time': 0.004168164, 'queue_time': 0.011220416, 'total_time': 0.055077255}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8a98a42-dbaf-4c61-bfef-c5290b46ec93-0', usage_metadata={'input_tokens': 107, 'output_tokens': 28, 'total_tokens': 135})

In [15]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"You told me your name is John!  \n\nDo you want to talk about something else?  Maybe you have a question about something, or you'd like to play a game? 😄  \n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's great to meet you! \n\nIs there anything you'd like to chat about or do today? 😄  I'm happy to answer questions, tell you a story, or even try to write you a poem! ✍️✨  What's on your mind? 💭  \n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John!  \n\nWe've already met, remember? 😊 \n\nIs there something else I can help you with?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [21]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [19]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away and I'll do my best to provide you with useful and informative answers.  \n\nWhat can I help you with today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 31, 'total_tokens': 89, 'completion_time': 0.105454545, 'prompt_time': 0.000342918, 'queue_time': 0.013799791, 'total_time': 0.105797463}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-61408b68-b42b-42c7-a540-9786ec24f8a6-0', usage_metadata={'input_tokens': 31, 'output_tokens': 58, 'total_tokens': 89})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away!  😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.000301779, 'queue_time': -0.000242649, 'total_time': 0.063938143}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c6fe9da-c0b8-49de-8c7e-adfc8bf9acb5-0', usage_metadata={'input_tokens': 31, 'output_tokens': 35, 'total_tokens': 66})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish, as you told me! 😊  Is there anything else I can help you with? \n'

In [26]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 👋 \n\nमुझे बहुत खुशी है तुम्हारे साथ बात करने के लिए। \n\nक्या मैं तुम्हारे लिए कुछ कर सकता हूँ?  😄\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! \n\nआप कैसे हैं? 😊  \n'

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [28]:
response.content

'आपका नाम कृष्ण है।  😄 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\dalal\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dalal\.conda\envs\langchain\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dalal\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see 

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, like your favorite ice cream flavor.  \n\nWhat's your favorite flavor? 😊🍦\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2?" \n'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nWould you like to tell me your name? 😊  \n\n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. \n\nIf you have a math problem you'd like help with, please ask! 😊  \n\n"